#### IMF Data

In [ ]:
import pandas as pd
import json
import requests

In [ ]:
response = requests.get('https://www.imf.org/external/datamapper/api/v1/NGDPD')
GDP_results = response.json()

GDP_results.get('values').get('NGDPD').get('USA')

In [ ]:
response = requests.get('https://www.imf.org/external/datamapper/api/v1/countries')
available_countries = response.json()

# Iterating over each key, value pair in the 'countries' dictionary
for c, details in available_countries.get('countries', {}).items():
    label = details.get('label') # Getting the value of 'label' key in the dictionary pointed by 'c'
    if label is not None:  # Check to ensure 'label' exists before printing
        print(label)

In [6]:
migration_df = pd.read_csv('IRS_county_to_county_migration_2017-2021.csv')
len(migration_df)

447621

In [14]:
unwanted_values = [
    'Other flows - Midwest',
    'Other flows - South',
    'Other flows - West',
    'Other flows - Northeast',
    'Foreign - APO/FPO ZIPs',
    'Foreign - Other flows'
]

adj_df_1 = migration_df[~migration_df['dest_county'].isin(unwanted_values)]
adj_df_1.head(50)

,year,dest_state_id,dest_county_id,orig_state_id,orig_county_id,dest_state,dest_county,returns,individuals,adj_gross_income
0,2017,1,1,96,0,AL,Autauga County Total Migration-US and Foreign,2400,5702,1.250690e+08
1,2017,1,1,97,0,AL,Autauga County Total Migration-US,2366,5573,1.226960e+08
2,2017,1,1,97,1,AL,Autauga County Total Migration-Same State,1516,3352,6.618700e+07
3,2017,1,1,97,3,AL,Autauga County Total Migration-Different State,850,2221,5.651000e+07
4,2017,1,1,98,0,AL,Autauga County Total Migration-Foreign,34,129,2.373000e+06
5,2017,1,1,1,1,AL,Autauga County Non-migrants,17484,39711,1.106647e+09
6,2017,1,1,1,51,AL,Elmore County,535,1155,2.366600e+07
7,2017,1,1,1,101,AL,Montgomery County,454,1039,1.979900e+07
8,2017,1,1,1,21,AL,Chilton County,83,220,3.006000e+06
9,2017,1,1,1,47,AL,Dallas County,82,195,2.917000e+06
